### We will first try to train and optimize some ML algorithms to predict the survival of individuals, using all available and meaningful data, 

### and then, following the data analysis, we search for the best algorithm only for predicting the survival of males, following the schema:
- if female:
    - if all family died:
        - predict die
    - else predict survive
- if male:
    - if all family survives:
        - predict survive
    - else predict survival using ML on data: IsYoung, PClass, and Embarked

In [1]:
using Pkg
Pkg.add(["CSV", "DataFrames", "Statistics", "MLJ", "MLJFlux", "Flux", "MLJLIBSVMInterface", "NearestNeighborModels"])

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
using CSV, DataFrames, Statistics, MLJ, MLJFlux, Flux, MLJLIBSVMInterface, NearestNeighborModels

### Read the data

In [3]:
data = CSV.File("data/train.csv") |> DataFrame
X_pred = CSV.File("data/test.csv") |> DataFrame;

### Let's create a pipeline for pre-processing the data
The columns we will use are:
* Sex
* IsYoung / CategoricalAge
* Pclass
* Embarked
* FamilySurvived

We will also have to change the scientific type of our data for it to work with the Julia ML algorithms.

In [4]:
# Function encodes the port name
function port_to_numerical(port)
    if ismissing(port) || port == "S"
        return 0 
    elseif port == "C"
        return 2
    else
        return 1
    end
end

port_to_numerical (generic function with 1 method)

In [5]:
# regex to extract first name
family_name_regex = r"^([\w\-]+)"

# Function returns a dictionary containing family names as keys, 
# and an array with the "Survived" label for all family members as value 
function get_family_dict(data)
    family_dict = Dict()
    
    # extract string
    for value in eachrow(data)
        name = match(family_name_regex, value.Name).match
        if haskey(family_dict, name)
            # push survived value to set matching the name 
            push!(get(family_dict, name, nothing), value.Survived)
        else
            # if dict does not contain this family, create a new set containing the "Survived" value of this person
            family_dict[name] = [value.Survived]
        end
    end
    
    return family_dict
end

get_family_dict (generic function with 1 method)

In [6]:
# function returns 0 if all family died
# - 1 if the person is travelling alone or family has mixed survival
# - 2 if all family survived
function family_survived(example, family_dict)
    name = match(family_name_regex, example.Name).match
    
    if haskey(family_dict, name) && length(get(family_dict, name, nothing)) > 1
        value = get(family_dict, name, nothing)
        if mean(value) == 1
            # all family survived
            return 2
        elseif mean(value) == 0
            # all family died
            return 0
        end
        # mixed survival
        return 1
    else
        # person is travelling alone
        return 1
    end
end

family_survived (generic function with 1 method)

In [7]:
# Function for pre-processing the data
# If given training data, only the "data" parameter has to be speciffied.
# For data to be predicted, the family dictionary must also be passed.
# @return family_dict only for training data.
function pre_process(data; train=true, family_dict=missing)
    # Sex to numerical
    data.Sex = map(x -> x == "male" ? 0 : 1, data.Sex)
    
    # IsYoung / Age
    # TODO
    data.Age = map(x -> !ismissing(x) && x < 14 ? 1 : 0, data.Age)
    rename!(data, :Age => :IsYoung)
    
    # Embarked to numerical
    data.Embarked = map(x -> port_to_numerical(x), data.Embarked)
    
    # FamilySurvived
    if train
        family_dict = get_family_dict(data)
    end
    data.FamilySurvived = map(x -> family_survived(x, family_dict), eachrow(data))
        
    # drop cols
    select!(data, Not([:PassengerId, :Name, :SibSp, :Parch, :Fare, :Ticket, :Cabin]))
    
    # scitype
    coerce!(data, Count => Continuous)
    if train
        coerce!(data, :Survived => OrderedFactor)
        return family_dict
    end
end

pre_process (generic function with 1 method)

In [8]:
family_dict = pre_process(data)
pre_process(X_pred, train=false, family_dict=family_dict);

### Our data now is in the right format. The next step is to break it into examples and labels.

In [9]:
y, X = unpack(data, ==(:Survived), x->true, rng=123);
# shuffle data with a seed to mentain a certain consistency between runs

In [10]:
train, test = partition(eachindex(y), 0.7, shuffle=true);

### Let's find models that can be fitted to our data

In [11]:
models(matching(X, y))

48-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :docstring, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :input_scitype, :target_scitype, :output_scitype), T} where T<:Tuple}:
 (name = AdaBoostClassifier, package_name = ScikitLearn, ... )
 (name = AdaBoostStumpClassifier, package_name = DecisionTree, ... )
 (name = BaggingClassifier, package_name = ScikitLearn, ... )
 (name = BayesianLDA, package_name = MultivariateStats, ... )
 (name = BayesianLDA, package_name = ScikitLearn, ... )
 (name = BayesianQDA, package_name = ScikitLearn, ... )
 (name = BayesianSubspaceLDA, package_name = MultivariateStats, ... )
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name 

In [12]:
function write_to_file(data, path::String)
    fw = open(path, "w")
    write(fw, "PassengerId,Survived\n")
    close(fw)
    fw = open(path, "a")
    index = 892
    for value in data
        write(fw, "$index,$value\n")
        index += 1
    end
    close(fw)
end

write_to_file (generic function with 1 method)

### Neural Networks time :D

### To get the best results, we must first normalize the data

In [13]:
stand = Standardizer()

Standardizer(
    features = Symbol[],
    ignore = false,
    ordered_factor = false,
    count = false) @704

In [103]:
Model = @load NeuralNetworkClassifier
# we'll use mini-batch for training the MLP for faster convergence
# also change the number of hidden neurons from the default 0 to 10 
#     to allow the network to learn the, possibly, more complex relations between features. 
model = Model(builder=MLJFlux.Short(n_hidden=10, σ=Flux.elu, dropout=0.2),
    epochs=200, batch_size=100,
    acceleration=CUDALibs());

import MLJFlux ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/zYlo3/src/loading.jl:168
┌ Warning: `acceleration isa CUDALibs` but no CUDA device (GPU) currently live. 
└ @ MLJFlux /home/ahautelman/.julia/packages/MLJFlux/AeMUx/src/classifier.jl:39


In [104]:
pipe = @pipeline stand model;

In [105]:
# define some ranges for the hyper-parameters that we want to optimize.
r = range(pipe, :(neural_network_classifier.lambda), lower=0.0, upper=10.0)
r2 = range(pipe, :(neural_network_classifier.alpha), lower=0.0, upper=4.0)
r3 = range(pipe, :(neural_network_classifier.builder.dropout), lower=0, upper=0.6)

typename(MLJBase.NumericRange)(Float64, :(neural_network_classifier.builder.dropout), ... )

In [106]:
model = TunedModel(model=pipe,
                   ranges = [r, r2, r3],
                   resampling=Holdout(fraction_train=0.8, shuffle=true),
                   measures=cross_entropy,
                   repeats=5);

In [107]:
mach = machine(model, X, y)
MLJ.fit!(mach, rows=train)

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…} @865.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Attempting to evaluate 1000 models.
└ @ MLJTuning /home/ahautelman/.julia/packages/MLJTuning/9sSuR/src/tuned_models.jl:564
Evaluating over 1000 metamodels: 100%[=========================] Time: 0:35:50
┌ Warning: `acceleration isa CUDALibs` but no CUDA device (GPU) currently live. 
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:436


Machine{ProbabilisticTunedModel{Grid,…},…} @865 trained 1 time; caches data
  args: 
    1:	Source @800 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @456 ⏎ `AbstractVector{OrderedFactor{2}}`


In [139]:
ŷ = predict(mach, X[test, :]);
misclassification_rate(mode.(ŷ), y[test])

0.14606741573033707

### Good score on the test set

In [138]:
prediction = convert(Array{Int64}, mode.(predict(mach, X_pred)));
write_to_file(prediction, "data/nn.csv")

### Kaggle score: 0.78708

In [141]:
MLJ.save("NN.jlso", mach)

### SVC algorithm

In [145]:
@load SVC
model = SVC()

import MLJLIBSVMInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/zYlo3/src/loading.jl:168


SVC(
    kernel = LIBSVM.Kernel.RadialBasis,
    gamma = 0.0,
    weights = nothing,
    cost = 1.0,
    cachesize = 200.0,
    degree = 3,
    coef0 = 0.0,
    tolerance = 0.001,
    shrinking = true,
    probability = false) @564

In [146]:
pipe = @pipeline stand model;

In [147]:
mach = machine(pipe, X, y)

Machine{Pipeline423,…} @698 trained 0 times; caches data
  args: 
    1:	Source @191 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @996 ⏎ `AbstractVector{OrderedFactor{2}}`


In [148]:
fit!(mach, rows=train)

┌ Info: Training Machine{Pipeline423,…} @698.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Training Machine{Standardizer,…} @476.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Training Machine{SVC,…} @949.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342


Machine{Pipeline423,…} @698 trained 1 time; caches data
  args: 
    1:	Source @191 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @996 ⏎ `AbstractVector{OrderedFactor{2}}`


In [149]:
ŷ = predict(mach, X[test, :])
misclassification_rate(ŷ, y[test])

0.13108614232209737

### Even better score for the SVC

In [153]:
prediction = convert(Array{Int64}, predict(mach, X_pred))
write_to_file(prediction, "data/svc.csv")

### Kaggle score: 0.78468

In [154]:
MLJ.save("SVC.jlso", mach)

### RandomForest classifier

In [182]:
Model = @load RandomForestClassifier pkg="DecisionTree"
model = Model()

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/zYlo3/src/loading.jl:168


RandomForestClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = -1,
    n_trees = 10,
    sampling_fraction = 0.7,
    pdf_smoothing = 0.0) @446

In [183]:
r2 = range(model, :min_samples_leaf, lower=1, upper=20)
r3 = range(model, :min_samples_split, lower=1, upper=40)
r4 = range(model, :n_subfeatures, lower=0, upper=5)
r5 = range(model, :n_trees, lower=2, upper=15)
r6 = range(model, :sampling_fraction, lower=0.5, upper=1.0);

In [184]:
model = TunedModel(model=model,
                  ranges=[r2, r3, r4, r5, r6],
                  resampling=Holdout(fraction_train=0.8, shuffle=true),
                  measures=cross_entropy,
                  repeats=5,
                  n=100000,
                  acceleration=CPUProcesses())

ProbabilisticTunedModel(
    model = RandomForestClassifier(
            max_depth = -1,
            min_samples_leaf = 1,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = -1,
            n_trees = 10,
            sampling_fraction = 0.7,
            pdf_smoothing = 0.0),
    tuning = Grid(
            goal = nothing,
            resolution = 10,
            shuffle = true,
            rng = Random._GLOBAL_RNG()),
    resampling = Holdout(
            fraction_train = 0.8,
            shuffle = true,
            rng = Random._GLOBAL_RNG()),
    measure = LogLoss(
            tol = 2.220446049250313e-16),
    weights = nothing,
    operation = MLJModelInterface.predict,
    range = MLJBase.NumericRange{T, MLJBase.Bounded, Symbol} where T[NumericRange{Int64,…} @286, NumericRange{Int64,…} @121, NumericRange{Int64,…} @141, NumericRange{Int64,…} @883, NumericRange{Float64,…} @909],
    selection_heuristic = MLJTuning.NaiveSelection(nothin

In [185]:
mach = machine(model, X, y);

In [186]:
fit!(mach, rows=train)

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…} @842.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Attempting to evaluate 100000 models.
└ @ MLJTuning /home/ahautelman/.julia/packages/MLJTuning/9sSuR/src/tuned_models.jl:564
Evaluating over 60000 metamodels: 100%[=========================] Time: 0:07:32
┌ Info: Only 60000 (of 100000) models evaluated.
│ Model supply exhausted. 
└ @ MLJTuning /home/ahautelman/.julia/packages/MLJTuning/9sSuR/src/tuned_models.jl:505


Machine{ProbabilisticTunedModel{Grid,…},…} @842 trained 1 time; caches data
  args: 
    1:	Source @112 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @544 ⏎ `AbstractVector{OrderedFactor{2}}`


In [187]:
ŷ = mode.(predict(mach, X[test, :]))
misclassification_rate(ŷ, y[test])

0.18352059925093633

In [191]:
prediction = convert(Array{Int64}, mode.(predict(mach, X_pred)))
write_to_file(prediction, "data/random_forest.csv")

### Kaggle score: 0.77751

In [192]:
MLJ.save("random_forest.jlso", mach)

### KNN clasiffier

In [89]:
@load KNNClassifier pkg="NearestNeighborModels"
model = KNNClassifier()

import NearestNeighborModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = Uniform()) @346

In [90]:
pipe = @pipeline stand model

Pipeline447(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    knn_classifier = KNNClassifier(
            K = 5,
            algorithm = :kdtree,
            metric = Euclidean(0.0),
            leafsize = 10,
            reorder = true,
            weights = Uniform())) @015

In [91]:
r = range(pipe, :(knn_classifier.K), values=2:40)

typename(MLJBase.NominalRange)(Int64, :(knn_classifier.K), ... )

In [92]:
model = TunedModel(model=pipe,
                   ranges=r,
                   resampling=Holdout(fraction_train=0.8, shuffle=true),
                   measures=cross_entropy,
                   repeats=5);

In [93]:
mach = machine(model, X, y)

Machine{ProbabilisticTunedModel{Grid,…},…} @122 trained 0 times; caches data
  args: 
    1:	Source @249 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @462 ⏎ `AbstractVector{OrderedFactor{2}}`


In [94]:
fit!(mach, rows=train)

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…} @122.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Attempting to evaluate 39 models.
└ @ MLJTuning /home/ahautelman/.julia/packages/MLJTuning/7mI9k/src/tuned_models.jl:564
Evaluating over 39 metamodels: 100%[=========================] Time: 0:00:04


Machine{ProbabilisticTunedModel{Grid,…},…} @122 trained 1 time; caches data
  args: 
    1:	Source @249 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @462 ⏎ `AbstractVector{OrderedFactor{2}}`


In [97]:
ŷ = mode.(MLJ.predict(mach, X[test, :]))
misclassification_rate(ŷ, y[test])

0.12734082397003746

In [230]:
prediction = convert(Array{Int64}, mode.(predict(mach, X_pred)))
write_to_file(prediction, "data/knn.csv")

### Kaggle score: 0.78947

In [98]:
MLJ.save("knn.jlso", mach)

### Light gbm ??

# TODO: use other approach

### As we have seen, the algorithms are all consistent and have good results.
But we got an accuracy increase of only 2% compared to the naive 'all women survive' prediction.

To change the tactics a bit, we will go by this next schema in the steps to come:
- if female:
    - if all family died:
        - predict die
    - else predict survival using ML
- if male:
    - if all family survives:
        - predict survive
    - else predict survival using ML algorithm


In [41]:
@load KNNClassifier pkg="NearestNeighborModels"
model = KNNClassifier()

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


import NearestNeighborModels ✔


KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = Uniform()) @395

In [42]:
r = range(model, :K, values=2:40)

typename(MLJBase.NominalRange)(Int64, :K, ... )

In [44]:
model = TunedModel(model=model,
                   ranges=r,
                   resampling=Holdout(fraction_train=0.8, shuffle=true),
                   measures=cross_entropy,
                   repeats=5);

In [46]:
mach = machine(model, X, y)
fit!(mach, rows=train)

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…} @802.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Attempting to evaluate 39 models.
└ @ MLJTuning /home/ahautelman/.julia/packages/MLJTuning/7mI9k/src/tuned_models.jl:564
Evaluating over 39 metamodels: 100%[=========================] Time: 0:00:19


Machine{ProbabilisticTunedModel{Grid,…},…} @802 trained 1 time; caches data
  args: 
    1:	Source @497 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @529 ⏎ `AbstractVector{OrderedFactor{2}}`


In [49]:
function my_predict(data, mach)
    ŷ = []
    for value in eachrow(data)
        push!(ŷ, predict_example(value, mach))
    end
    return ŷ
end

my_predict (generic function with 1 method)

In [130]:
function predict_example(example, mach)
    if example.FamilySurvived == 0
        return 0    
    elseif example.FamilySurvived == 2
        return 1
    else
        pred = predict(mach, DataFrame(example))
        # have to do this dumb shit cus julia has problems with predicting only 1 example
        for d in pred
            if pdf(d, 0.0) >= 0.5
                return 0
            else
                return 1
            end
        end
    end
end

predict_example (generic function with 1 method)

In [134]:
prediction = my_predict(X_pred, mach)

418-element Vector{Any}:
 0
 1
 0
 0
 1
 0
 1
 1
 1
 0
 0
 0
 1
 ⋮
 0
 0
 1
 1
 1
 1
 1
 0
 1
 0
 0
 1

In [135]:
write_to_file(prediction, "knn_stack.csv")

In [126]:
@doc pdf

```
pdf(d::UnivariateDistribution, x::Real)
```

Evaluate the probability density (mass) at `x`.

See also: [`logpdf`](@ref).

```
pdf(d::MultivariateDistribution, x::AbstractArray)
```

Return the probability density of distribution `d` evaluated at `x`.

  * If `x` is a vector, it returns the result as a scalar.
  * If `x` is a matrix with n columns, it returns a vector `r` of length n, where `r[i]` corresponds

to `x[:,i]` (i.e. treating each column as a sample).

`pdf!(r, d, x)` will write the results to a pre-allocated array `r`.

```
pdf(d::MatrixDistribution, x::AbstractArray)
```

Compute the probability density at the input matrix `x`.

```
pdf(d::Union{UnivariateMixture, MultivariateMixture}, x)
```

Evaluate the (mixed) probability density function over `x`. Here, `x` can be a single sample or an array of multiple samples.

```
Distributions.pdf(d::UnivariateFinite, x)
```

Probability of `d` at `x`.

```
v = categorical(["yes", "maybe", "no", "yes"])
d = UnivariateFinite(v[1:2], [0.3, 0.7])
pdf(d, "yes")     # 0.3
pdf(d, v[1])      # 0.3
pdf(d, "no")      # 0.0
pdf(d, "house")   # throws error
```

Other similar methods are available too:

```
mode(d)    # CategoricalValue{String, UInt32} "maybe"
rand(d, 5) # CategoricalArray{String,1,UInt32}["maybe", "no", "maybe", "maybe", "no"] or similar
d = fit(UnivariateFinite, v)
pdf(d, "maybe") # 0.25
logpdf(d, "maybe") # log(0.25)
```

One can also do weighted fits:

```
w = [1, 4, 5, 1] # some weights
d = fit(UnivariateFinite, v, w)
pdf(d, "maybe") ≈ 4/11 # true
```

See also `classes`, `support`.


# TODO: create pipeline for predicting